In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

def generate_samples(label, n, center):
    """Generates synthetic pluvial data centered around certain conditions."""
    return pd.DataFrame({
        "rainfall_intensity": np.random.normal(loc=center["rainfall"], scale=12, size=n),
        "impervious_ratio":   np.random.normal(loc=center["impervious"], scale=0.08, size=n),
        "drainage_density":   np.random.normal(loc=center["drainage"], scale=0.4, size=n),
        "urbanization_index": np.random.normal(loc=center["urban"], scale=0.08, size=n),
        "convergence_index":  np.random.normal(loc=center["convergence"], scale=0.1, size=n),
        "pluvial_binary":     label
    })

# --- Core Archetypes ---
flood_urban = generate_samples(1, 1000, {
    "rainfall": 45, "impervious": 0.8, "drainage": 1.5, "urban": 0.85, "convergence": 0.75
})

flood_rural_rain = generate_samples(1, 500, {
    "rainfall": 90, "impervious": 0.3, "drainage": 2.5, "urban": 0.35, "convergence": 0.85
})

no_flood_urban_dry = generate_samples(0, 700, {
    "rainfall": 15, "impervious": 0.8, "drainage": 3.5, "urban": 0.8, "convergence": 0.6
})

no_flood_rural = generate_samples(0, 800, {
    "rainfall": 35, "impervious": 0.25, "drainage": 4.2, "urban": 0.25, "convergence": 0.35
})

contrast_mix = generate_samples(1, 300, {
    "rainfall": 60, "impervious": 0.5, "drainage": 2.8, "urban": 0.6, "convergence": 0.55
})

data = pd.concat([flood_urban, flood_rural_rain, contrast_mix, no_flood_urban_dry, no_flood_rural])
data = data.clip(lower=0, upper=None)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

data["rainfall_intensity"] *= 2

data.to_csv("dataset/pluvial_flood_data_balanced.csv", index=False)
print("✅ Pluvial dataset rebuilt:", data.shape)


✅ Pluvial dataset rebuilt: (3300, 6)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.saving import register_keras_serializable
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# --- Load Pluvial Dataset ---
data = pd.read_csv("dataset/pluvial_flood_data_balanced.csv")

print("✅ Pluvial dataset size:", len(data),
      "| Flood:", data['pluvial_binary'].sum(),
      "| No Flood:", (data['pluvial_binary'] == 0).sum())

X = data.drop("pluvial_binary", axis=1).astype("float32")
y = data["pluvial_binary"].astype("float32")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42)

@register_keras_serializable()
def surface_runoff_amplifier(inputs):
    rain = inputs[:, 0]
    impervious = inputs[:, 1]
    rain_boost = tf.sigmoid((rain - 60) * 0.06)
    impervious_boost = tf.sigmoid((impervious - 0.6) * 10)
    return (1.0 + 0.3 * rain_boost * impervious_boost)[:, None]

@register_keras_serializable()
def drainage_penalty(inputs):
    dd = inputs[:, 2]
    return (1.0 - 0.4 * tf.sigmoid((dd - 3.5) * 2))[:, None]

@register_keras_serializable()
def convergence_suppressor(inputs):
    ci = inputs[:, 4]
    return (1.0 + 0.3 * tf.sigmoid((ci - 0.5) * 8))[:, None]

@register_keras_serializable()
def clip_modulation(x):
    return tf.clip_by_value(x, 0.7, 1.3)

input_layer = layers.Input(shape=(5,))

rain_input = layers.Lambda(lambda x: x[:, 0:1])(input_layer)
rain_branch = layers.Dense(8, activation="relu")(rain_input)

# --- Main trunk with residual connection ---
x = layers.BatchNormalization()(input_layer)
x1 = layers.Dense(128, activation="relu")(x)
x2 = layers.Dense(64, activation="relu")(x1)
x3 = layers.Dense(64, activation="relu")(x2)

residual = layers.Add()([x3, x2])  # ← residual connection
combined = layers.Concatenate()([residual, rain_branch])
logits = layers.Dense(1)(combined)


amplifier    = layers.Lambda(surface_runoff_amplifier)(input_layer)
penalty      = layers.Lambda(drainage_penalty)(input_layer)
suppression  = layers.Lambda(convergence_suppressor)(input_layer)

mod_strength = layers.Multiply()([amplifier, penalty, suppression])
mod_strength = layers.Lambda(clip_modulation)(mod_strength)

modulated_logits = layers.Add()([logits, mod_strength])
adjusted_output  = layers.Activation("sigmoid")(modulated_logits)

model = models.Model(inputs=input_layer, outputs=adjusted_output)
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.05),
    metrics=["accuracy"]
)

# --- Train & Evaluate ---
early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=8, callbacks=[early_stop])

loss, acc = model.evaluate(X_test, y_test)
print(f"🌧️ PluvialNet Accuracy: {acc:.4f}")
model.save("models/PV-FloodNet.h5")


✅ Pluvial dataset size: 3300 | Flood: 1800 | No Flood: 1500
Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.8445 - loss: 0.3536 - val_accuracy: 0.9827 - val_loss: 0.1705
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9669 - loss: 0.1941 - val_accuracy: 0.9892 - val_loss: 0.1609
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9760 - loss: 0.1806 - val_accuracy: 0.9957 - val_loss: 0.1425
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9650 - loss: 0.2045 - val_accuracy: 0.9935 - val_loss: 0.1437
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9676 - loss: 0.1955 - val_accuracy: 0.9957 - val_loss: 0.1472
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9795 - loss: 0.1773 - val_accuracy: 0.9935 - val_loss: 0.1483
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9728 - loss: 0.1753 - val_accuracy: 0.9935 - val_loss: 0.1452
Epoch 8/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 1s 

🌧️ PluvialNet Accuracy: 0.9960
